# EDA 5: Pokročilejší manipulace a statistika

* Práce s indexy
* Segmentace, agregace, groupby, pivot
* Elementární optimalizace (%timeit)
* Pokročilejší vizualizace se seaborn
* Základy statistických testů
* Data: IMDB+...?


https://github.com/alanderex/pydata-pandas-workshop/tree/master/notebooks

https://github.com/TomAugspurger/pandas-head-to-tail/blob/master/notebooks/03-Iterators-Groupby.ipynb

In [1]:
import pandas as pd

In [2]:
import seaborn as sns

In [3]:
import plotly_express as px

In [4]:
sns.set_context("notebook")
sns.set_style("whitegrid")
sns.set_palette("hls")

In [5]:
%matplotlib inline
# This will enable us to see plots embedded in the noteboo

# Analýza dat po skupinách

Jedním ze základních postupů v datové analýze je rozdělení do skupin, aplikace nějaké oprace na jednotlivé skupiny a nakonec kombinace výsledků do vhodného datasetu. Anglicky se tento postup označuje jako *split-apply-combine*. Skupiny jsou často definovány nějakou (kategorickou) proměnnou, např. by to mohla být barva, pohlaví nebo kontinent. Skupiny lze ale vytvářet i odvozováním, např. pomocí rozsahu nebo nebo vlastností časových řad. Oboje už jsme vlastně viděli v EDA 3, kdy jsme skupiny vytvářeli pro deštivé dny nebo jednotlivé roky.

Pojďme si to vysvětlit prakticky. Použijeme k tomu hezký dataset se údaji z amerického kongresu.

In [6]:
# odkaz přímo na csv soubor na internetu
LEGISLATORS_HISTORICAL_URL_CSV = (
    "https://theunitedstates.io/congress-legislators/legislators-historical.csv"
)

In [7]:
# některé sloupce uložíme rovnou jako kategorické
dtypes = {
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}

legislators_historical = pd.read_csv(
    LEGISLATORS_HISTORICAL_URL_CSV,
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "first_name", "last_name"],
    parse_dates=["birthday"],
)

In [8]:
legislators_historical

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11976,Collins,Chris,1950-05-20,M,rep,NY,Republican
11977,Cummings,Elijah,1951-01-18,M,rep,MD,Democrat
11978,Hill,Katie,1987-08-25,F,rep,CA,Democrat
11979,Isakson,John,1944-12-28,M,sen,GA,Republican


### Krok 1: Split

Na rozdělení dat do skupin slouží metoda [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby). Nejjednodušší a možná i nejčastější použití je seskupení podle existujícího sloupce v tabulce. Třeba podle `type`, tedy u nás konkrétně podle komory: `rep` je sněmovna reprezentatntů, `sen` je senát.

In [9]:
legislators_historical_by_type = legislators_historical.groupby("type")
legislators_historical_by_type

Dostali jsme objekt typu [`GroupBy`](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html). Pokud vám to připomíná `DatetimeIndexResampler` z [Explorativní analýzy a statistiky jedné proměnné](https://naucse.python.cz/2020/pydata-praha-jaro/pydata/eda-univariate-timeseries/), je to velice dobrá asociace. Tato operace totiž data jen rozdělila do skupin, ještě jsme ale neřekli co s těmi skupinami chceme dělat.

### Krok 2 + 3: Apply + Combine

Na skupiny, které jsme vytvořili pomocí `groupby`, teď můžeme aplikovat nějakou opraci. To je právě krok *apply*. Nemá pro nás moc cenu oddělovat tento krok od třetího kroku *combine*, protože když už nějakou opraci aplikujeme, tak bychom rádi výsledek sestavili do výsledného datasetu. Pandas navíc tyto kroky sám nijak neodděluje.

*Pozn.: Apply a combine začne být více odděleno v nástrojích na zpracování vělkých dat, které už se nevejdou pohodlně do operační paměti počítače a Pandas na ně už nestačí. Apply pak probíhá po částech, třeba i distribuovaně na oddělených serverech, a výsledné combine se provádí sesbíráním částečných výsledků.* 

Použijeme teď jednoduchou agregační metodu `count`, která nám vrátí počet hodnot (po skupinách samozřejmě).

In [10]:
legislators_historical_by_type.count()

,last_name,first_name,birthday,gender,state,party
type,,,,,,
rep,10151,10151,9658,10151,10151,9925
sen,1830,1830,1770,1830,1830,1822


Vidíme, že v datech je zaznamenáno 1830 senátorů a 10151 kongresmanů. U některých chybí údaje o datu narození nebo straně.

**Úkol:** Rozděl data podle strany (`party`) a vypiš počet záznamů v každé skupině. Dokážeš výsledek setřídit podle velikosti skupin? 

In [12]:
# řešení
legislators_historical.groupby(["party"]).count().sort_values("last_name", ascending=False).head(5)

,last_name,first_name,birthday,gender,type,state
party,,,,,,
Democrat,5005,5005,4913,5005,5005,5005
Republican,4881,4881,4701,4881,4881,4881
Whig,564,564,508,564,564,564
Federalist,343,343,286,343,343,343
Jackson,338,338,263,338,338,338


Možná jste si všimly/i, jaký je u výsledku index. Pokud ne, nevadí, určitě si všimnete teď. Zkusíme totiž vytvořit skupiny ne z jednoho sloupce, ale ze dvou. Pojďme si rozdělit zákonodárce podle států, a každou skupinu za jeden stát ještě podle pohlaví.

In [46]:
legislators_by_state_gender_counts = legislators_historical.groupby(["state", "gender"]).count()
legislators_by_state_gender_counts

last_name  first_name  birthday   type  party
state gender                                               
AK    F             NaN         NaN       NaN    NaN    NaN
      M            16.0        16.0      16.0   16.0   14.0
AL    F             3.0         3.0       3.0    3.0    3.0
      M           203.0       203.0     194.0  203.0  203.0
AR    F             5.0         5.0       5.0    5.0    5.0
...                 ...         ...       ...    ...    ...
WI    M           197.0       197.0     197.0  197.0  197.0
WV    F             1.0         1.0       1.0    1.0    1.0
      M           119.0       119.0     117.0  119.0  119.0
WY    F             2.0         2.0       2.0    2.0    2.0
      M            38.0        38.0      38.0   38.0   38.0

[116 rows x 5 columns]

Máme tedy skupiny, které jsou definované dvojicí hodnot stát a pohlaví (`state`, `gender`). A to je přesně důvod, proč existuje v Pandas [`MultiIndex`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html).

### Práce s MultiIndex

Vlastnosti MultiIndexu, vlastně takového víceúrovňového či vícerozměrného indexu, můžeme prozkoumat (kromě prostého zobrazení) pomocí několika užitečných atributů (properties).

In [47]:
# počet úrovní 
legislators_by_state_gender_counts.index.nlevels

2

In [48]:
# jména úrovní
legislators_by_state_gender_counts.index.names

FrozenList(['state', 'gender'])

In [49]:
# mohutnost (počet hodnot) jednotlivých úrovní
legislators_by_state_gender_counts.index.levshape

(58, 2)

In [50]:
# hodnoty v jednotlivých úrovních
legislators_by_state_gender_counts.index.levels

FrozenList([['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'DK', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OL', 'OR', 'PA', 'PI', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'], ['F', 'M']])

Víme tedy, že náš (multi) index má dvě úrovně. Abychom dostali konkrétní řádek, musíme tím pádem zadat dvě hodnoty. K tomu nám poslouží `tuple` (pozor, musí to opravdu být `tuple` a ne `list`, tj. musíme použít kulaté a ne hranaté závorky).

In [51]:
legislators_by_state_gender_counts.loc[("WY", "F")]

last_name     2.0
first_name    2.0
birthday      2.0
type          2.0
party         2.0
Name: (WY, F), dtype: float64

Co kdybychom zadali jen polovinu indexu? Dostaneme celou skupinu, v našem případě celý stát.

In [52]:
legislators_by_state_gender_counts.loc["WY"]

,last_name,first_name,birthday,type,party
gender,,,,,
F,2.0,2.0,2.0,2.0,2.0
M,38.0,38.0,38.0,38.0,38.0


**Otázka:** Jaký je index výsledné tabulky?

Pokud bychom chtěli jedno pohlaví, můžeme indexu změnit pořadí.

In [54]:
swapped_index = legislators_by_state_gender_counts.index.swaplevel(0, 1)
legislators_by_gender_state_counts = legislators_by_state_gender_counts.set_index(swapped_index)
legislators_by_gender_state_counts

,,last_name,first_name,birthday,type,party
gender,state,,,,,
F,AK,NaN,NaN,NaN,NaN,NaN
M,AK,16.0,16.0,16.0,16.0,14.0
F,AL,3.0,3.0,3.0,3.0,3.0
M,AL,203.0,203.0,194.0,203.0,203.0
F,AR,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...
M,WI,197.0,197.0,197.0,197.0,197.0
F,WV,1.0,1.0,1.0,1.0,1.0
M,WV,119.0,119.0,117.0,119.0,119.0


In [56]:
legislators_by_gender_state_counts.loc["F"].head()

,last_name,first_name,birthday,type,party
state,,,,,
AK,NaN,NaN,NaN,NaN,NaN
AL,3.0,3.0,3.0,3.0,3.0
AR,5.0,5.0,5.0,5.0,5.0
AS,NaN,NaN,NaN,NaN,NaN
AZ,3.0,3.0,3.0,3.0,3.0


Více o (pokročileší) práci s indexi a multiindexi najdeš v [dokumentaci](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Pokud bychom se chtěli multiindexu "zbavit" můžeme to udělat pomocí `.reset_index()`

In [38]:
legislators_by_state_gender_counts.reset_index().head(5)

,state,gender,last_name,first_name,birthday,type,party
0,AK,F,NaN,NaN,NaN,NaN,NaN
1,AK,M,16.0,16.0,16.0,16.0,14.0
2,AL,F,3.0,3.0,3.0,3.0,3.0
3,AL,M,203.0,203.0,194.0,203.0,203.0
4,AR,F,5.0,5.0,5.0,5.0,5.0


Anebo rovnou použít `groupby` s `as_index=False`.

In [39]:
legislators_historical.groupby(["state", "gender"], as_index=False).count().head(5)

,state,gender,last_name,first_name,birthday,type,party
0,AK,F,NaN,NaN,NaN,NaN,NaN
1,AK,M,16.0,16.0,16.0,16.0,14.0
2,AL,F,3.0,3.0,3.0,3.0,3.0
3,AL,M,203.0,203.0,194.0,203.0,203.0
4,AR,F,5.0,5.0,5.0,5.0,5.0


**Úkol:** TODO

## Jak  `groupby` funguje

Je užitečné si ukázat, že `groupby` vlastně není v principu nijak složité. Začneme s vytvořením ` GroupBy` objektu podle států.

In [57]:
legislators_by_state = legislators_historical.groupby(["state"])
legislators_by_state

Prvním krokem ve zkoumání vnitřností `GroupBy` bude ukázka, jak skupiny iterovat. To může být v praxi užitečné třeba pro interaktivní zkoumání dat ve skupinách. 

In [59]:
# itertools.islice nám pomůže vzít ve for cyklu jen první tři skupiny
import itertools

for group_name, group_data in itertools.islice(legislators_by_state, 3):
    print(f"--- Sample of group '{group_name}' ---")
    print(group_data.sample(5))

--- Sample of group 'AK' ---
      last_name first_name   birthday gender type state       party
11386   Stevens        Ted 1923-11-18      M  sen    AK  Republican
10325    Gravel    Maurice 1930-05-13      M  sen    AK    Democrat
10108    Begich   Nicholas 1932-04-06      M  rep    AK    Democrat
11734    Begich       Mark 1962-03-30      M  sen    AK    Democrat
9951     Rivers      Ralph 1903-05-23      M  rep    AK    Democrat
--- Sample of group 'AL' ---
     last_name first_name   birthday gender type state       party
991     Walker       John 1783-08-12      M  sen    AL  Republican
6172   Aldrich    William 1853-03-11      M  rep    AL  Republican
4331    Rapier      James 1837-11-13      M  rep    AL  Republican
5093  Williams     Thomas 1825-08-11      M  rep    AL    Democrat
5821     Oates    William 1835-11-30      M  rep    AL    Democrat
--- Sample of group 'AR' ---
       last_name first_name   birthday gender type state       party
10356      Mills     Wilbur 1909-0

Jednotlivé skupiny můžeme získat i přímo - `GroupBy` se totiž chová trochu jako slovník. Názvy skupin získáme pomocí `.keys()`

In [60]:
legislators_by_state.groups.keys()

dict_keys(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'DK', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OL', 'OR', 'PA', 'PI', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'])

A skupinu pomocí `[ ]` indexace:

In [61]:
legislators_by_state.groups["AK"]

Int64Index([ 6619,  6647,  7442,  7501,  8039,  8236,  8877,  9819,  9951,
             9985, 10082, 10108, 10325, 11262, 11386, 11734],
           dtype='int64')

Vlastně ne tak docela - získali jsme indexy. Ty můžeme samozřejmě použít s `.loc`, tj. 
```python
legislators_by_state.loc[legislators_by_state.groups["AK"]]
```
anebo využijeme zkratku přes `get_group`:

In [62]:
legislators_by_state.get_group("AK")

,last_name,first_name,birthday,gender,type,state,party
6619,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6647,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7442,Grigsby,George,1874-12-02,M,rep,AK,NaN
7501,Sulzer,Charles,1879-02-24,M,rep,AK,NaN
8039,Sutherland,Daniel,1869-04-17,M,rep,AK,Republican
8236,Wickersham,James,1857-08-24,M,rep,AK,Republican
8877,Dimond,Anthony,1881-11-30,M,rep,AK,Democrat
9819,Gruening,Ernest,1887-02-06,M,sen,AK,Democrat
9951,Rivers,Ralph,1903-05-23,M,rep,AK,Democrat
9985,Bartlett,Edward,1904-04-20,M,sen,AK,Democrat


Na skupinu můžeme použít nějakou agregaci, např. naše oblíbené `count`, a máme hotový krok 2. apply:

In [63]:
legislators_by_state.get_group("AK").count()

last_name     16
first_name    16
birthday      16
gender        16
type          16
state         16
party         14
dtype: int64

A nakonec můžeme vše zkombinovat třeba přes slovník, ze kterého vytvoříme tabulku.

In [67]:
pd.DataFrame.from_dict(
    {
        group_name: group_items["last_name"].count()
        for group_name, group_items in legislators_by_state
    },
    orient="index",
)

,0
AK,16
AL,206
AR,117
AS,2
AZ,48
CA,363
CO,90
CT,240
DC,2
DE,97


### TOHLE ASI NAKONEC NE

In [68]:
LEGISLATORS_HISTORICAL_URL_JSON = (
    "https://theunitedstates.io/congress-legislators/legislators-historical.json"
)

In [69]:
import pandas as pd
import json

with open("legislators-historical.json", "r") as f:
    df = pd.io.json.json_normalize(json.load(f))

In [71]:
df.columns

Index(['terms', 'id.bioguide', 'id.govtrack', 'id.icpsr', 'id.wikipedia',
       'id.wikidata', 'id.google_entity_id', 'name.first', 'name.last',
       'bio.birthday', 'bio.gender', 'id.house_history', 'name.middle',
       'name.nickname', 'name.suffix', 'id.bioguide_previous',
       'id.house_history_alternate', 'other_names', 'id.thomas', 'id.cspan',
       'id.votesmart', 'id.lis', 'id.ballotpedia', 'name.official_full',
       'id.opensecrets', 'id.fec', 'id.maplight', 'leadership_roles',
       'family'],
      dtype='object')

Vypočítej věk při prvním nástupu do sněmovny. Jaký je průměr, největší, nejmenší ve státě / komoře / státu / století? Rozděl na skupiny mladí / střední věk / staří a srovnej statistiky.

# Pokročilejší group by

Zatím jsme viděli groupby operace s jedním nebo dvěma existujícími sloupci a jednou agregací (`count`). Pojďme se podívat, jaké další možnosti groupby nabízí.

Na pomoc si vezmeme již známá data o počasí v Praze-Ruzyni.

In [72]:
RUZYNE_DATA_FILENAME = "P1PRUZ01.xls"

In [73]:
def extract_and_clean_chmi_excel_sheet(excel_data, sheet_name):
    """Parse ČHMÚ historical meteo excel data"""
    # načti list z excel souboru a převeď na tidy data formát
    data_tidy = (
        excel_data.parse(sheet_name, skiprows=3)
        .melt(id_vars=["rok", "měsíc"], var_name="den", value_name=sheet_name)
        .dropna()
    )
    # vytvoř časovou řadu datumů
    datum = pd.to_datetime(
        data_tidy[["rok", "měsíc", "den"]].rename(
            columns={"rok": "year", "měsíc": "month", "den": "day"}
        )
    )
    # přidej sloupec datum jako index a odstraň den, měsíc, rok a vrať setříděný výsledek
    return (
        data_tidy.assign(datum=datum)
        .set_index("datum")
        .drop(columns=["rok", "měsíc", "den"])
        .sort_index()
    )

In [74]:
# otevři Excel soubor
excel_data_ruzyne = pd.ExcelFile(RUZYNE_DATA_FILENAME)
# načti všechny listy kromě prvního
extracted_sheets = (
    extract_and_clean_chmi_excel_sheet(excel_data_ruzyne, sheet_name)
    for sheet_name in excel_data_ruzyne.sheet_names[1:]
)
# spoj všechny listy do jednoho DataFrame
ruzyne_tidy = pd.concat(extracted_sheets, axis=1)

In [75]:
ruzyne_tidy

,teplota průměrná,teplota maximální,teplota minimální,rychlost větru,tlak vzduchu,vlhkost vzduchu,úhrn srážek,celková výška sněhu,sluneční svit
datum,,,,,,,,,
1961-01-01,-2.0,1.0,-3.4,1.7,969.9,87.0,0.0,6.0,0.4
1961-01-02,-1.9,-1.2,-2.4,3.0,965.2,89.0,0.0,4.0,0.0
1961-01-03,0.1,1.0,-2.6,2.7,952.3,80.0,0.0,3.0,0.0
1961-01-04,-0.3,2.1,-2.0,3.3,953.6,87.0,0.0,3.0,1.0
1961-01-05,0.4,2.8,-4.3,6.3,963.9,81.0,0.0,2.0,2.2
...,...,...,...,...,...,...,...,...,...
2018-12-27,4.0,6.5,2.7,5.4,981.4,88.0,0.0,0.0,0.5
2018-12-28,3.9,6.2,0.3,3.9,981.5,82.0,0.0,0.0,3.9
2018-12-29,2.2,4.8,1.3,5.8,983.8,87.0,0.8,0.0,0.5


Už dříve jsme pomocí `resample` dokázali vytvořit roční statistiky. To samé můžeme udělat i pomocí `groupby`. Jen nepoužijeme existující sloupec (museli bychom uměle vytvořit sloupec s rokem) a použijeme sloupec s kategoriemi (roky) rovnou jako argument `groupby`.

In [76]:
ruzyne_tidy.groupby(ruzyne_tidy.index.year).mean().head()

,teplota průměrná,teplota maximální,teplota minimální,rychlost větru,tlak vzduchu,vlhkost vzduchu,úhrn srážek,celková výška sněhu,sluneční svit
datum,,,,,,,,,
1961,8.616986,13.440822,4.202740,3.482740,971.669863,77.265753,1.225479,0.183562,4.872329
1962,7.113425,11.660548,2.908493,4.051233,971.157260,75.975342,1.175890,0.342466,4.926027
1963,7.182466,11.950411,2.827945,3.384658,970.853973,76.558904,1.407397,3.298630,5.060548
1964,7.761475,12.227322,3.423224,3.838525,972.823497,75.931694,1.250820,1.144809,4.929508
1965,7.164384,11.663562,3.135890,4.111507,969.060274,81.797260,1.759178,2.435616,4.525479


Co kdybychom ale chtěli nějaké statistiky pro typy dnů podle teploty, ještě rozdělené podle kvartálů? Už víme, že groupby může vytvářet skupinu podle více kritérií. Na více agregací pak můžeme použít `.agg(["mean", "max", "min"])`.

Nejprve ale vytvoříme kategorie dní podle teploty pomocí [`cut`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html).

In [114]:
temperature_bin_flags = pd.cut(
    ruzyne_tidy["teplota průměrná"], bins=3, labels=("cool", "warm", "hot"),
)
temperature_bin_flags

datum
1961-01-01    warm
1961-01-02    warm
1961-01-03    warm
1961-01-04    warm
1961-01-05    warm
              ... 
2018-12-27    warm
2018-12-28    warm
2018-12-29    warm
2018-12-30    warm
2018-12-31    warm
Name: teplota průměrná, Length: 21184, dtype: category
Categories (3, object): [cool < warm < hot]

Pokud chceme vidět explicitně hranice, použijeme `retbins=True`.

In [118]:
temperature_bin_flags, temperature_bins = pd.cut(
    ruzyne_tidy["teplota průměrná"], bins=3, labels=("cool", "warm", "hot"),
    retbins=True,
)
temperature_bins

array([-22.6521    ,  -5.23333333,  12.13333333,  29.5       ])

In [119]:
ruzyne_tidy.groupby([ruzyne_tidy.index.quarter, temperature_bin_flags]).agg(["mean", "max", "min"])

teplota průměrná             teplota maximální        \
                                   mean   max   min              mean   max   
datum teplota průměrná                                                        
1     cool                    -8.852374  -5.3 -22.6         -4.815579   3.4   
      warm                     1.800640  12.1  -5.2          5.346754  24.1   
      hot                     12.993750  15.8  12.2         19.287500  23.0   
2     cool                          NaN   NaN   NaN               NaN   NaN   
      warm                     8.043845  12.1  -3.0         13.010202  22.7   
      hot                     16.446180  27.4  12.2         22.310668  33.6   
3     cool                          NaN   NaN   NaN               NaN   NaN   
      warm                    10.318570  12.1   4.9         15.100376  20.8   
      hot                     17.628068  29.5  12.2         23.525975  37.4   
4     cool                    -8.339706  -5.3 -19.8         -4.811765   2.0   
      warm                     3.856536  12.1  -5.2          7.098287  21.1   
      hot                     13.828869  19.2  12.2         19.082143  25.4   

                             teplota minimální             rychlost větru  \
                         min              mean   max   min           mean   
datum teplota průměrná                                                      
1     cool             -18.3        -12.686944  -5.8 -25.5       3.208754   
      warm              -4.5         -1.497041  10.4 -17.1       4.902208   
      hot               13.9          5.687500  10.8   0.6       5.250000   
2     cool               NaN               NaN   NaN   NaN            NaN   
      warm              -0.7          3.712352  11.4  -7.2       4.573693   
      hot               13.2          9.947454  20.0   0.1       3.621886   
3     cool               NaN               NaN   NaN   NaN            NaN   
      warm               7.0          6.845169  11.4  -0.5       4.290339   
      hot               13.4         11.967592  20.7   1.7       3.567592   
4     cool             -17.5        -11.602941  -6.0 -23.5       3.127941   
      warm              -4.6          0.883989  11.2 -16.5       4.390398   
      hot               12.7          9.047024  15.8   2.2       4.202381   

                        ... vlhkost vzduchu úhrn srážek             \
                        ...             min        mean   max  min   
datum teplota průměrná  ...                                          
1     cool              ...            53.0    0.353264  12.0  0.0   
      warm              ...            37.0    0.844037  24.9  0.0   
      hot               ...            42.0    0.918750   6.8  0.0   
2     cool              ...             NaN         NaN   NaN  NaN   
      warm              ...            33.0    1.746501  49.2  0.0   
      hot               ...            32.0    2.058500  74.5  0.0   
3     cool              ...             NaN         NaN   NaN  NaN   
      warm              ...            54.0    2.036136  93.3  0.0   
      hot               ...            29.0    1.917493  56.5  0.0   
4     cool              ...            59.0    0.268382   9.0  0.0   
      warm              ...            42.0    1.012690  30.6  0.0   
      hot               ...            49.0    0.756548  16.9  0.0   

                       celková výška sněhu            sluneční svit             
                                      mean   max  min          mean   max  min  
datum teplota průměrná                                                          
1     cool                        9.627596  46.0  0.0      3.494214  10.6  0.0  
      warm                        2.346290  57.0  0.0      2.773940  12.3  0.0  
      hot                         0.000000   0.0  0.0      6.896875  11.8  0.0  
2     cool                             NaN   NaN  NaN           NaN   NaN  NaN  
      warm                        0.071669  20.0  0.0      4.89

Ještě napravíme jména sloupců v (multi) indexu.

In [120]:
ruzyne_tidy.groupby([ruzyne_tidy.index.quarter, temperature_bin_flags]).agg(
    ["mean", "max", "min"]
).rename_axis(["quarter", "temp_bin"])

teplota průměrná             teplota maximální              \
                             mean   max   min              mean   max   min   
quarter temp_bin                                                              
1       cool            -8.852374  -5.3 -22.6         -4.815579   3.4 -18.3   
        warm             1.800640  12.1  -5.2          5.346754  24.1  -4.5   
        hot             12.993750  15.8  12.2         19.287500  23.0  13.9   
2       cool                  NaN   NaN   NaN               NaN   NaN   NaN   
        warm             8.043845  12.1  -3.0         13.010202  22.7  -0.7   
        hot             16.446180  27.4  12.2         22.310668  33.6  13.2   
3       cool                  NaN   NaN   NaN               NaN   NaN   NaN   
        warm            10.318570  12.1   4.9         15.100376  20.8   7.0   
        hot             17.628068  29.5  12.2         23.525975  37.4  13.4   
4       cool            -8.339706  -5.3 -19.8         -4.811765   2.0 -17.5   
        warm             3.856536  12.1  -5.2          7.098287  21.1  -4.6   
        hot             13.828869  19.2  12.2         19.082143  25.4  12.7   

                 teplota minimální             rychlost větru  ...  \
                              mean   max   min           mean  ...   
quarter temp_bin                                               ...   
1       cool            -12.686944  -5.8 -25.5       3.208754  ...   
        warm             -1.497041  10.4 -17.1       4.902208  ...   
        hot               5.687500  10.8   0.6       5.250000  ...   
2       cool                   NaN   NaN   NaN            NaN  ...   
        warm              3.712352  11.4  -7.2       4.573693  ...   
        hot               9.947454  20.0   0.1       3.621886  ...   
3       cool                   NaN   NaN   NaN            NaN  ...   
        warm              6.845169  11.4  -0.5       4.290339  ...   
        hot              11.967592  20.7   1.7       3.567592  ...   
4       cool            -11.602941  -6.0 -23.5       3.127941  ...   
        warm              0.883989  11.2 -16.5       4.390398  ...   
        hot               9.047024  15.8   2.2       4.202381  ...   

                 vlhkost vzduchu úhrn srážek            celková výška sněhu  \
                             min        mean   max  min                mean   
quarter temp_bin                                                              
1       cool                53.0    0.353264  12.0  0.0            9.627596   
        warm                37.0    0.844037  24.9  0.0            2.346290   
        hot                 42.0    0.918750   6.8  0.0            0.000000   
2       cool                 NaN         NaN   NaN  NaN                 NaN   
        warm                33.0    1.746501  49.2  0.0            0.071669   
        hot                 32.0    2.058500  74.5  0.0            0.000000   
3       cool                 NaN         NaN   NaN  NaN                 NaN   
        warm                54.0    2.036136  93.3  0.0            0.000000   
        hot                 29.0    1.917493  56.5  0.0            0.000000   
4       cool                59.0    0.268382   9.0  0.0            6.900735   
        warm                42.0    1.012690  30.6  0.0            0.661168   
        hot                 49.0    0.756548  16.9  0.0            0.000000   

                            sluneční svit             
                   max  min          mean   max  min  
quarter temp_bin                                      
1       cool      46.0  0.0      3.494214  10.6  0.0  
        warm      57.0  0.0      2.773940  12.3  0.0  
        hot        0.0  0.0      6.896875  11.8  0.0  
2       cool       NaN  NaN           NaN   NaN  NaN  
        warm      20.0  0.0      4.897681  15.0  0.0  
        hot        0.0  0.0      8.521370  15.7  0.0  
3       cool       NaN  NaN           NaN   NaN  NaN  
        warm       0.0  0.0      3.986324  1

Můžu si také vybrat jen některé sloupce a agregace.

In [122]:
ruzyne_by_quarter_temperature = (
    ruzyne_tidy.groupby([ruzyne_tidy.index.quarter, temperature_bin_flags])
    .agg(
        {
            "teplota průměrná": "mean",
            "teplota maximální": "max",
            "teplota minimální": "min",
            "sluneční svit": ["max", "mean"],
        }
    )
    .rename_axis(["quarter", "temp_bin"])
)
ruzyne_by_quarter_temperature

teplota průměrná teplota maximální teplota minimální  \
                             mean               max               min   
quarter temp_bin                                                        
1       cool            -8.852374               3.4             -25.5   
        warm             1.800640              24.1             -17.1   
        hot             12.993750              23.0               0.6   
2       cool                  NaN               NaN               NaN   
        warm             8.043845              22.7              -7.2   
        hot             16.446180              33.6               0.1   
3       cool                  NaN               NaN               NaN   
        warm            10.318570              20.8              -0.5   
        hot             17.628068              37.4               1.7   
4       cool            -8.339706               2.0             -23.5   
        warm             3.856536              21.1             -16.5   
        hot             13.828869              25.4               2.2   

                 sluneční svit            
                           max      mean  
quarter temp_bin                          
1       cool              10.6  3.494214  
        warm              12.3  2.773940  
        hot               11.8  6.896875  
2       cool               NaN       NaN  
        warm              15.0  4.897681  
        hot               15.7  8.521370  
3       cool               NaN       NaN  
        warm              13.5  3.986324  
        hot               15.7  7.210377  
4       cool               7.3  2.861765  
        warm              11.0  2.207128  
        hot               11.1  4.594643

Kromě multiindexu pro řádky dostáváme i multiindex pro sloupce-

In [123]:
ruzyne_by_quarter_temperature.columns

MultiIndex([( 'teplota průměrná', 'mean'),
            ('teplota maximální',  'max'),
            ('teplota minimální',  'min'),
            (    'sluneční svit',  'max'),
            (    'sluneční svit', 'mean')],
           )

In [124]:
ruzyne_by_quarter_temperature.unstack()

teplota průměrná                       teplota maximální              \
                     mean                                     max               
temp_bin             cool       warm        hot              cool  warm   hot   
quarter                                                                         
1               -8.852374   1.800640  12.993750               3.4  24.1  23.0   
2                     NaN   8.043845  16.446180               NaN  22.7  33.6   
3                     NaN  10.318570  17.628068               NaN  20.8  37.4   
4               -8.339706   3.856536  13.828869               2.0  21.1  25.4   

         teplota minimální            sluneční svit                        \
                       min                      max                  mean   
temp_bin              cool  warm  hot          cool  warm   hot      cool   
quarter                                                                     
1                    -25.5 -17.1  0.6          10.6  12.3  11.8  3.494214   
2                      NaN  -7.2  0.1           NaN  15.0  15.7       NaN   
3                      NaN  -0.5  1.7           NaN  13.5  15.7       NaN   
4                    -23.5 -16.5  2.2           7.3  11.0  11.1  2.861765   

                              
                              
temp_bin      warm       hot  
quarter                       
1         2.773940  6.896875  
2         4.897681  8.521370  
3         3.986324  7.210377  
4         2.207128  4.594643

*Nápad:* Statistický test - jsou víkendy studenější? Svítí víc v létě?

*TODO:* `transform`, `filter`, `apply` ???

## Pivoting - TODO

> pivot (third-person singular simple present pivots, present participle pivoting, simple past and past participle pivoted)
 **To turn on an exact spot.**
 
> A pivot table is a table of statistics that summarizes the data of a more extensive table ...
> Although pivot table is a generic term, Microsoft Corporation trademarked PivotTable in the United States in 1994.

Our pivoting task: Get a table with numbers of titles per year (as row) and type (as column).

One approach is to use `groupby`, `count` aggregation and `unstack`.

In [ ]:
grouped_by_year_and_type = imdb_data.groupby(["startYear", "titleType"])

In [ ]:
pivoted = grouped_by_year_and_type.numVotes.count().unstack()
pivoted.tail()

There's a shortcut though, see if you we can use it.

**Exercise:** Create the `pivoted` table using `pivot_table`:

In [ ]:
%exercise

pivot_table = imdb_data.pivot_table(values=___, index=___, columns=___, aggfunc=___)

# display - do not edit
pivot_table.tail()

In [ ]:
%validate

pd.testing.assert_frame_equal(pivoted, pivot_table)

We can now use this to plot a kind of a histogram with colour for title types.

In [ ]:
with sns.color_palette("Paired"):
    fig, ax = plt.subplots(figsize=(16, 6))
    pivoted.loc[1990:].plot.bar(stacked=True, ax=ax)

## Final mini-project - creative, unbounded, free-style

Here are some ideas of what you can do with the data.

* Create 5-star rating based on quantiles using `quantile` and `cut` or `qcut`.
* Group by studio / decade / rating
* Compare simple (arithmetic) mean `averageRating` in each group with `averageRating` average weighted by `numVotes` ($ \frac{\sum \rm{averageRating} \times \rm{numVotes}} {\sum \rm{numVotes}} $). Use `apply` and the `wavg` function from https://pbpython.com/weighted-average.html. This function is quite time and memory consuming and thus not ideal for large data sets. You can try to implement weighted average using standard `mean`. Check the performance with the `%timeit` magic.
* Use the 5-star rating for `hue` in an interesting seaborn plot (see https://seaborn.pydata.org/tutorial/relational.html)
* Use `sns.catplot` to visualize the distrubution of incomes in each 5-star rating group. 

A couple more ideas can be found in https://github.com/brandon-rhodes/pycon-pandas-tutorial

After you have solved all of those, come up with your own quests - we may still be around and help you :-D